In [ ]:
import csv
import boto3
from decimal import Decimal

def lambda_handler(event, context):
    # Check if the user_date slot is populated
    if 'currentIntent' in event and 'slots' in event['currentIntent'] and 'user_date' in event['currentIntent']['slots']:
        user_date = event['currentIntent']['slots']['user_date']
    else:
        return {
            'dialogAction': {
                'type': 'Close',
                'fulfillmentState': 'Failed',
                'message': {
                    'contentType': 'PlainText',
                    'content': 'Invalid event structure. Missing user_date slot.'
                }
            }
        }
    
    # Define the S3 bucket and file name
    bucket_name = 'project2fintech2023'
    file_name = 'total_chart_save.csv'
    
    # Create an S3 client
    s3 = boto3.client('s3')
    
    try:
        # Get the CSV file from S3
        response = s3.get_object(Bucket=bucket_name, Key=file_name)
        csv_data = response['Body'].read().decode('utf-8').splitlines()
        
        # Read the CSV data
        reader = csv.reader(csv_data)
        next(reader)  # Skip the header row
        
        # Iterate through the rows and find the matching date
        for row in reader:
            date = row[0]
            if date == user_date:
                predicted_price = Decimal(row[1])
                formatted_price = f" ${predicted_price:.2f} CAD"
                return {
                    'dialogAction': {
                        'type': 'Close',
                        'fulfillmentState': 'Fulfilled',
                        'message': {
                            'contentType': 'PlainText',
                            'content': f"The predicted composite benchmark for {user_date} is {formatted_price}."
                        }
                    }
                }
        
        # If the date is not found, return an error message
        return {
            'dialogAction': {
                'type': 'Close',
                'fulfillmentState': 'Failed',
                'message': {
                    'contentType': 'PlainText',
                    'content': f"No data found for the specified date {user_date}."
                }
            }
        }
    
    except Exception as e:
        return {
            'dialogAction': {
                'type': 'Close',
                'fulfillmentState': 'Failed',
                'message': {
                    'contentType': 'PlainText',
                    'content': f"Error retrieving the CSV file: {str(e)}"
                }
            }
        }
